In [1]:
import numpy as np
import csv
import os
import os.path as path
from tqdm import tqdm, trange
import cv2
import time
import copy # For SeqNMS

In [79]:
pred_ext = '_all.csv'
path_pred = '/Users/lucas/Desktop/test_mAP_2'
path_image = '/Users/lucas/Desktop/test_mAP_2'

path_nms = '/Users/lucas/Desktop/yolo_test_pred'

classes = ["Person_sitting", "Cyclist", "Pedestrian", "Van", "Truck", "Misc", "DontCare", "Car", "Tram"]

display_result = False

# Standard NMS
std_obj_threshold = 0.3
std_iou_threshold = 0.3

# Soft NMS (Linear)
soft_lin_obj_threshold = 0.3
soft_lin_iou_threshold = 0.3

# Soft NMS (Gaussian)
soft_gaus_obj_threshold = 0.3
soft_gaus_iou_threshold = 0.3
soft_gaus_sigma = 0.4

# Seq NMS

# Soft Seq NMS

In [3]:
def import_pred(file_path):
    with open(file_path) as csvfile:
        csvreader = csv.reader(csvfile, delimiter=',')
        list_row = [row for row in csvreader]
        row_count = len(list_row)
        col_count = len(list_row[0])

        bboxes = np.zeros((row_count, col_count))

        for idx, row in enumerate(list_row):
            bboxes[idx,:] = np.asarray([float(num) for num in row])

    return bboxes

In [4]:
def convert_array_to_pred(bboxes, im_shape, classes):
    pred_list = []
    num_boxes = bboxes.shape[0]
    
    for idx in range(num_boxes):
        if np.sum(bboxes[idx, 5:]) > 0:
            x, y, w, h = bboxes[idx, :4]
            
            xmin  = int((x - w/2) * im_shape[1])
            xmax  = int((x + w/2) * im_shape[1])
            ymin  = int((y - h/2) * im_shape[0])
            ymax  = int((y + h/2) * im_shape[0])
            label = classes[np.argmax(bboxes[idx, 5:])].encode("utf-8")
            score = np.max(bboxes[idx, 5:])
            pred_list.append([label, xmin, ymin, xmax, ymax, score])
    
    return pred_list

In [61]:
def nms_fast(boxes, iou_threshold, obj_threshold, sigma=0.5, method=''):
    '''
    Inspired from: https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/ Malisiewicz et al.
    boxes = [x, y, w, h, classe_score] all need to be float
    '''
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
 
    # grab the coordinates of the bounding boxes and the score for the class
    x1 = boxes[:,0] - boxes[:,2] /2
    y1 = boxes[:,1] - boxes[:,3] /2
    x2 = boxes[:,0] + boxes[:,2] /2  # because boxes[:,2] is w
    y2 = boxes[:,1] + boxes[:,3] /2  # because boxes[:,3] is h
    sc = boxes[:,4]
    
    sc_output = np.zeros(boxes.shape[0])
    
    if np.max(sc) < obj_threshold: # Test if some boxes need to be considered
        return [], [], sc_output
    else:
        # initialize the list of picked indexes
        pick = []
        
        # compute the area of the bounding boxes and sort the bounding
        # boxes by the bottom-right y-coordinate of the bounding box
        area = (x2 - x1) * (y2 - y1)
        idxs = np.argsort(sc)
        
        # remove the boxes of which the score is already too small
        id_to_remove = np.nonzero(sc < obj_threshold)
        idxs = idxs[~np.in1d(idxs,id_to_remove)]

        # keep looping while some indexes still remain in the indexes
        # list
        while len(idxs) > 0:
            # grab the last index in the indexes list and add the
            # index value to the list of picked indexes
            i = idxs[-1]
            pick.append(i)
            sc_output[i] = sc[i]

            # Remove the index picked up from the list of index to explore
            idxs = idxs[:-1]
            
            # find the largest (x, y) coordinates for the start of
            # the bounding box and the smallest (x, y) coordinates
            # for the end of the bounding box
            xx1 = np.maximum(x1[i], x1[idxs])
            yy1 = np.maximum(y1[i], y1[idxs])
            xx2 = np.minimum(x2[i], x2[idxs])
            yy2 = np.minimum(y2[i], y2[idxs])

            # compute the width and height of the bounding box
            w = np.maximum(0, xx2 - xx1)
            h = np.maximum(0, yy2 - yy1)

            # compute the ratio of overlap
            iou = (w * h) / (area[idxs] + area[i] - (w * h))

            # reducce the score of the ones that have overlap
            if method == 'linear':
                sc[idxs] *= 1 - (iou * (iou > iou_threshold))
            elif method == 'gaussian':
                sc[idxs] *= np.exp(-(iou * iou * (iou > iou_threshold))/sigma)
            else: # traditional method
                sc[idxs] *= iou < iou_threshold

            # delete all indexes for which the score is too low
            id_to_remove = np.nonzero(sc < obj_threshold)
            idxs = idxs[~np.in1d(idxs,id_to_remove)]
        
    # return only the bounding boxes that were picked using the
    # integer data type
    return boxes[pick], pick, sc_output

In [62]:
def nms_multiclass(org_boxes, iou_threshold, obj_threshold, sigma=0.5, method=''):
    
    boxes = np.copy(org_boxes)
    
    nb_class = len(boxes[0][5:])
    
    for idx in range(nb_class):
        # Isolate bounding boxes and specific class
        class_array = np.concatenate((boxes[:,:4], boxes[:,[5+idx]]), axis=1)
        
        # Run NMS algorithm for selected class
        _, _, new_cls_scr = nms_fast(class_array, std_iou_threshold, std_obj_threshold, sigma=sigma, method=method)
        
        # Replace score by new scores
        boxes[:, 5+idx] = new_cls_scr
    
    return boxes

In [72]:
# -*- coding: utf-8 -*-
# import _init_paths
# import numpy as np
# from fast_rcnn.nms_wrapper import nms
# import cv2
# import time
# import copy
# import cPickle as pickle
# import os

# CLASSES = ('__background__',
#            'airplane','antelope','bear','bicycle','bird','bus',
#            'car','cattle','dog','domestic cat','elephant','fox',
#            'giant panda','hamster','horse','lion','lizard','monkey',
#            'motorcycle','rabbit','red panda','sheep','snake','squirrel',
#            'tiger','train','turtle','watercraft','whale','zebra')

#CONF_THRESH = 0.5
#NMS_THRESH = 0.3
IOU_THRESH = 0.6

'''
修改检测结果格式，用作后续处理
第一维：种类
第二维：帧
第三维：bbox
第四维：x1,y1,x2,y2,score
'''
# def createInputs(video):
#     create_begin=time.time()
#     frames=sorted(video.keys()) #获得按序排列的帧的名称
#     dets=[[] for i in CLASSES[1:]] #保存最终结果
#     for cls_ind,cls in enumerate(CLASSES[1:]): #类
#         for frame_ind,frame in enumerate(frames): #帧
#             cls_boxes = video[frame]['boxes'][:, 4*(cls_ind+1):4*(cls_ind + 2)]
#             cls_scores = video[frame]['scores'][:, cls_ind+1]
#             cls_dets = np.hstack((cls_boxes,cls_scores[:, np.newaxis])).astype(np.float64)
#             dets[cls_ind].append(cls_dets)
#     create_end=time.time()
#     print 'create inputs: {:.4f}s'.format(create_end - create_begin)
#     return dets

def createLinks(dets_all, classes):
    links_all=[]
    #建立每相邻两帧之间的link关系
    frame_num=len(dets_all[0])
    cls_num=len(classes)-1
    #links_all=[] #保存每一类的全部link，第一维为类数，第二维为帧数-1，为该类下的links即每一帧与后一帧之间的link，第三维每帧的box数，为该帧与后一帧之间的link
    for cls_ind in range(cls_num): #第一层循环，类数
        links_cls=[] #保存一类下全部帧的links
        link_begin=time.time()
        for frame_ind in range(frame_num-1): #第二层循环，帧数-1，不循环最后一帧
            dets1=dets_all[cls_ind][frame_ind]
            dets2=dets_all[cls_ind][frame_ind+1]
            box1_num=len(dets1)
            box2_num=len(dets2)
            #先计算每个box的area
            if frame_ind==0:
                areas1=np.empty(box1_num)
                for box1_ind,box1 in enumerate(dets1):
                    areas1[box1_ind]=(box1[2]-box1[0]+1)*(box1[3]-box1[1]+1)
            else: #当前帧的area1就是前一帧的area2，避免重复计算
                areas1=areas2
            areas2=np.empty(box2_num)
            for box2_ind,box2 in enumerate(dets2):
                areas2[box2_ind]=(box2[2]-box2[0]+1)*(box2[3]-box2[1]+1)
            #计算相邻两帧同一类的link
            links_frame=[] #保存相邻两帧的links
            for box1_ind,box1 in enumerate(dets1):
                area1=areas1[box1_ind]
                x1=np.maximum(box1[0],dets2[:,0])
                y1=np.maximum(box1[1],dets2[:,1])
                x2=np.minimum(box1[2],dets2[:,2])
                y2=np.minimum(box1[3],dets2[:,3])
                w =np.maximum(0.0, x2 - x1 + 1)
                h =np.maximum(0.0, y2 - y1 + 1)
                inter = w * h
                ovrs = inter / (area1 + areas2 - inter)
                links_box=[ovr_ind for ovr_ind,ovr in enumerate(ovrs) if ovr >= IOU_THRESH] #保存第一帧的一个box对第二帧全部box的link
                links_frame.append(links_box)
            links_cls.append(links_frame)
        link_end=time.time()
        # print 'link: {:.4f}s'.format(link_end - link_begin)
        links_all.append(links_cls)
    return links_all

def maxPath(dets_all,links_all):
    for cls_ind,links_cls in enumerate(links_all):
        max_begin=time.time()
        dets_cls=dets_all[cls_ind]
        while True:
            rootindex,maxpath,maxsum=findMaxPath(links_cls,dets_cls)
            if len(maxpath) <= 1:
                break
            rescore(dets_cls,rootindex,maxpath,maxsum)
            deleteLink(dets_cls,links_cls,rootindex,maxpath,IOU_THRESH)
        max_end=time.time()
        # print 'max path: {:.4f}s'.format(max_end - max_begin)

# def NMS(dets_all):
#     for cls_ind,dets_cls in enumerate(dets_all):
#         for frame_ind,dets in enumerate(dets_cls):
#             keep=nms(dets, NMS_THRESH)
#             dets_all[cls_ind][frame_ind]=dets[keep, :]

def findMaxPath(links,dets):
    maxpaths=[] #保存从每个结点到最后的最大路径与分数
    roots=[] #保存所有的可作为独立路径进行最大路径比较的路径
    maxpaths.append([ (box[4],[ind]) for ind,box in enumerate(dets[-1])])
    for link_ind,link in enumerate(links[::-1]): #每一帧与后一帧的link，为一个list
        curmaxpaths=[]
        linkflags=np.zeros(len(maxpaths[0]),int)
        det_ind=len(links)-link_ind-1
        for ind,linkboxes in enumerate(link): #每一帧中每个box的link，为一个list
            if linkboxes == []:
                curmaxpaths.append((dets[det_ind][ind][4],[ind]))
                continue
            linkflags[linkboxes]=1
            prev_ind=np.argmax([maxpaths[0][linkbox][0] for linkbox in linkboxes])
            prev_score=maxpaths[0][linkboxes[prev_ind]][0]
            prev_path=copy.copy(maxpaths[0][linkboxes[prev_ind]][1])
            prev_path.insert(0,ind)
            curmaxpaths.append((dets[det_ind][ind][4]+prev_score,prev_path))
        root=[maxpaths[0][ind] for ind,flag in enumerate(linkflags) if flag == 0]
        roots.insert(0,root)
        maxpaths.insert(0,curmaxpaths)
    roots.insert(0,maxpaths[0])
    maxscore=0
    maxpath=[]
    for index,paths in enumerate(roots):
        if paths==[]:
            continue
        maxindex=np.argmax([path[0] for path in paths])
        if paths[maxindex][0]>maxscore:
            maxscore=paths[maxindex][0]
            maxpath=paths[maxindex][1]
            rootindex=index
    return rootindex,maxpath,maxscore

def rescore(dets, rootindex, maxpath, maxsum):
    newscore=maxsum/len(maxpath)
    for i,box_ind in enumerate(maxpath):
        dets[rootindex+i][box_ind][4]=newscore

def deleteLink(dets,links, rootindex, maxpath,thesh):
    for i,box_ind in enumerate(maxpath):
        areas=[(box[2]-box[0]+1)*(box[3]-box[1]+1) for box in dets[rootindex+i]]
        area1=areas[box_ind]
        box1=dets[rootindex+i][box_ind]
        x1=np.maximum(box1[0],dets[rootindex+i][:,0])
        y1=np.maximum(box1[1],dets[rootindex+i][:,1])
        x2=np.minimum(box1[2],dets[rootindex+i][:,2])
        y2=np.minimum(box1[3],dets[rootindex+i][:,3])
        w =np.maximum(0.0, x2 - x1 + 1)
        h =np.maximum(0.0, y2 - y1 + 1)
        inter = w * h
        ovrs = inter / (area1 + areas - inter)
        deletes=[ovr_ind for ovr_ind,ovr in enumerate(ovrs) if ovr >= IOU_THRESH] #保存待删除的box的index
        if rootindex+i<len(links): #除了最后一帧，置box_ind的box的link为空
            for delete_ind in deletes:
                links[rootindex+i][delete_ind]=[]
        if i > 0 or rootindex>0:
            for priorbox in links[rootindex+i-1]: #将前一帧指向box_ind的link删除
                for delete_ind in deletes:
                    if delete_ind in priorbox:
                        priorbox.remove(delete_ind)

# def dsnms(video):
#     dets=createInputs(video)
#     links=createLinks(dets)
#     maxPath(dets,links)
#     NMS(dets)
#     frame_names=sorted(video.keys())
#     saveforAP(dets,frame_names)

# pkllistfile=open('/workspace/liruiguang/imagenet/pkllist.txt')
# pkllist=pkllistfile.readlines()
# pkllistfile.close()
# pkllist=[pkl.strip() for pkl in pkllist]
# for pkl in pkllist:
#     f = open('/workspace/liruiguang/imagenet/'+pkl)
#     video = pickle.load(f)
#     dsnms(video['dets'])

In [73]:
'''
修改检测结果格式，用作后续处理
第一维：种类
第二维：帧
第三维：bbox
第四维：x1,y1,x2,y2,score
'''
def createInputs(seq_boxes, classes, im_shape):
    #seq_boxes is a list of np_array
    dets=[[] for i in classes] #保存最终结果
    for cls_ind, cls in enumerate(classes): #类
        for boxes_ind, boxes in enumerate(seq_boxes): #帧
            
            cls_boxes = np.zeros((boxes.shape[0], 4))
            
            cls_boxes[:, 0] = (boxes[:,0] - boxes[:,2] /2) * im_shape[1]
            cls_boxes[:, 1] = (boxes[:,1] - boxes[:,3] /2) * im_shape[0]
            cls_boxes[:, 2] = (boxes[:,0] + boxes[:,2] /2) * im_shape[1]
            cls_boxes[:, 3] = (boxes[:,1] + boxes[:,3] /2) * im_shape[0]
            
            cls_scores = boxes[:,5+cls_ind]
            cls_dets = np.hstack((cls_boxes,cls_scores[:, np.newaxis])).astype(np.float64)
            dets[cls_ind].append(cls_dets)
   
    return dets

In [74]:
def seq_nms_rescore(seq_boxes, classes, im_shape):
    dets = createInputs(seq_boxes, classes, im_shape)
    links = createLinks(dets, classes)
    maxPath(dets,links)
    
    last_dets = [classes_boxes[-1] for classes_boxes in dets]
    
    boxes_rescored = np.zeros((last_dets[0].shape[0], 4 + 1 + len(last_dets))) # +1 for object confidence score
    boxes_rescored[:,:4] = seq_boxes[-1][:,:4] # Get original coordinates
    
    for idx, boxes_class in enumerate(last_dets):
        boxes_rescored[:, 5 +idx] = boxes_class[:,4]
    
    return boxes_rescored

In [75]:
def all_non_max_sup(all_boxes, std_iou_threshold, std_obj_threshold,\
                        soft_lin_iou_threshold, soft_lin_obj_threshold,\
                        soft_gaus_iou_threshold, soft_gaus_obj_threshold, soft_gaus_sigma):
        
    # --- Standard NMS ---
    start_time_std = time.time()
    nms_boxes = nms_multiclass(all_boxes, std_iou_threshold, std_obj_threshold, method='standard')
    time_std = time.time() - start_time_std


    # --- Soft NMS (Linear)---
    start_time_soft_lin = time.time()
    soft_lin_boxes = nms_multiclass(all_boxes, soft_lin_iou_threshold, soft_lin_obj_threshold,method='linear')
    time_soft_lin = time.time() - start_time_soft_lin


    # --- Soft NMS (Gaussian)---
    start_time_soft_gaus = time.time()
    soft_gaus_boxes = nms_multiclass(all_boxes, soft_gaus_iou_threshold, soft_gaus_obj_threshold,sigma=soft_gaus_sigma ,method='gaussian')
    time_soft_gaus = time.time() - start_time_soft_gaus
    
    
    return nms_boxes, soft_lin_boxes, soft_gaus_boxes, [time_std, time_soft_lin, time_soft_gaus]
    

In [76]:
def save_pred(dir_path, filename, pred, im_shape, classes):
    
    if not path.isdir(dir_path):
        os.makedirs(dir_path)
    
    csv_path = path.join(dir_path, filename + '.csv')
    
    pred_list = convert_array_to_pred(pred, im_shape, classes)
    
    with open(csv_path, "w") as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerows(pred_list)

In [77]:
# Import list predictions
list_file = [filename[:-len(pred_ext)] for filename in os.listdir(path_pred) if filename.endswith(pred_ext)]

print(len(list_file), ' files to convert.')
print(list_file)

(5, ' files to convert.')
['00025', '00037', '00008', '00089', '00028']


In [78]:
all_time = np.zeros((len(list_file), 7))

# Convert all the predictions
for idx in trange(len(list_file)):
    filename = list_file[idx]
    filepath = path.join(path_pred, filename + pred_ext)

    image = cv2.imread(path.join(path_image, filename + '.png'))

    # Import all bboxes
    all_boxes = import_pred(filepath)
    
    nms_boxes, soft_lin_boxes, soft_gaus_boxes, all_time[idx, :3] = all_non_max_sup(all_boxes, std_iou_threshold, std_obj_threshold,\
                                                                       soft_lin_iou_threshold, soft_lin_obj_threshold,\
                                                                       soft_gaus_iou_threshold, soft_gaus_obj_threshold, soft_gaus_sigma)
    # Save predictions to disk
    path_std_nms = path.join(path_nms, 'std_nms')
    save_pred(path_std_nms, filename, nms_boxes, image.shape, classes)
    
    path_soft_lin_nms = path.join(path_nms, 'soft_lin_nms')
    save_pred(path_soft_lin_nms, filename, soft_lin_boxes, image.shape, classes)
    
    path_soft_gaus_nms = path.join(path_nms, 'soft_gaus_nms')
    save_pred(path_soft_gaus_nms, filename, soft_gaus_boxes, image.shape, classes)
    
    # Rescore boxes with Seq-NMS
    # TODO import previous prediction
    # TODO create list of predictions
    
    start_time_seq_rescore = time.time()
    boxes_rescored = seq_nms_rescore([all_boxes], classes, image.shape)
    all_time[idx, 3] = time.time() - start_time_seq_rescore
    
    seq_nms_boxes, seq_soft_lin_boxes, seq_soft_gaus_boxes, all_time[idx, 4:7] = all_non_max_sup(boxes_rescored, std_iou_threshold, std_obj_threshold,\
                                                                       soft_lin_iou_threshold, soft_lin_obj_threshold,\
                                                                       soft_gaus_iou_threshold, soft_gaus_obj_threshold, soft_gaus_sigma)
    all_time[idx, 4:7] += all_time[idx, 3]
    
     # Save predictions to disk
    path_seq_std_nms = path.join(path_nms, 'seq_std_nms')
    save_pred(path_seq_std_nms, filename, seq_nms_boxes, image.shape, classes)
    
    path_seq_soft_lin_nms = path.join(path_nms, 'seq_soft_lin_nms')
    save_pred(path_seq_soft_lin_nms, filename, seq_soft_lin_boxes, image.shape, classes)
    
    path_seq_soft_gaus_nms = path.join(path_nms, 'seq_soft_gaus_nms')
    save_pred(path_seq_soft_gaus_nms, filename, seq_soft_gaus_boxes, image.shape, classes)

    # Display the results
    if display_result:
        std_pred             = convert_array_to_pred(nms_boxes, image.shape, classes)
        soft_lin_pred        = convert_array_to_pred(soft_lin_boxes, image.shape, classes)
        soft_gaus_pred       = convert_array_to_pred(soft_gaus_boxes, image.shape, classes)
        seq_std_pred         = convert_array_to_pred(seq_nms_boxes, image.shape, classes)
        seq_soft_lin_pred    = convert_array_to_pred(seq_soft_lin_boxes, image.shape, classes)
        seq_soft_gaus_pred   = convert_array_to_pred(seq_soft_gaus_boxes, image.shape, classes)

        print('Image: ' + filename)
        print('Standard NMS:')
        print(std_pred)
        print('Soft NMS (Linear):')
        print(soft_lin_pred)
        print('Soft NMS (Gaussian):')
        print(soft_gaus_pred)
        print('Seq + Standard NMS:')
        print(seq_std_pred)
        print('Seq + Soft NMS (Linear):')
        print(seq_soft_lin_pred)
        print('Seq + Soft NMS (Gaussian):')
        print(seq_soft_gaus_pred)
        print('-------------------')

# Display average time
print ('Average Time:')
print(' - Standard NMS              : ' + str(np.mean(all_time[:,0])))
print(' - Soft NMS (Linear)         : ' + str(np.mean(all_time[:,1])))
print(' - Soft NMS (Gaussian)       : ' + str(np.mean(all_time[:,2])))
print(' - Seq Rescore               : ' + str(np.mean(all_time[:,3])))
print(' - Seq + Standard NMS        : ' + str(np.mean(all_time[:,4])))
print(' - Seq + Soft NMS (Linear)   : ' + str(np.mean(all_time[:,5])))
print(' - Seq + Soft NMS (Gaussian) : ' + str(np.mean(all_time[:,6])))

 40%|████      | 2/5 [00:00<00:00,  8.12it/s]

Image: 00025
Standard NMS:
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
Soft NMS (Linear):
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
Soft NMS (Gaussian):
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
Seq + Standard NMS:
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
Seq + Soft NMS (Linear):
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
Seq + Soft NMS (Gaussian):
[['Car', 327, 212, 462, 285, 0.82588523626327515]]
-------------------
Image: 00037
Standard NMS:
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
Soft NMS (Linear):
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
Soft NMS (Gaussian):
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
Seq + Standard NMS:
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
Seq + Soft NMS (Linear):
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
Seq + Soft NMS (Gaussian):
[['Truck', 383, 125, 503, 240, 0.78033816814422607]]
-------------------


 80%|████████  | 4/5 [00:00<00:00,  7.66it/s]

Image: 00008
Standard NMS:
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
Soft NMS (Linear):
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
Soft NMS (Gaussian):
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
Seq + Standard NMS:
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
Seq + Soft NMS (Linear):
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
Seq + Soft NMS (Gaussian):
[['Car', 885, 179, 1176, 306, 0.83759605884552002]]
-------------------
Image: 00089
Standard NMS:
[['Car', 764, 181, 869, 236, 0.30477333068847656], ['Car', 858, 160, 1103, 294, 0.93076246976852417], ['Car', 290, 200, 460, 286, 0.77619814872741699]]
Soft NMS (Linear):
[['Car', 764, 181, 869, 236, 0.30477333068847656], ['Car', 858, 160, 1103, 294, 0.93076246976852417], ['Car', 290, 200, 460, 286, 0.77619814872741699]]
Soft NMS (Gaussian):
[['Car', 764, 181, 869, 236, 0.30477333068847656], ['Car', 858, 160, 1103, 294, 0.93076246976852417], ['Car', 290, 200, 460, 286, 0.77619814872741699]]
Seq + 

100%|██████████| 5/5 [00:00<00:00,  7.30it/s]

Image: 00028
Standard NMS:
[]
Soft NMS (Linear):
[]
Soft NMS (Gaussian):
[]
Seq + Standard NMS:
[]
Seq + Soft NMS (Linear):
[]
Seq + Soft NMS (Gaussian):
[]
-------------------
Average Time:
 - Standard NMS              : 0.00139937400818
 - Soft NMS (Linear)         : 0.00127921104431
 - Soft NMS (Gaussian)       : 0.00153684616089
 - Seq Rescore               : 0.0121973991394
 - Seq + Standard NMS        : 0.0137330055237
 - Seq + Soft NMS (Linear)   : 0.0134789466858
 - Seq + Soft NMS (Gaussian) : 0.0134124755859
